<a href="https://colab.research.google.com/github/seamusrobertmurphy/01-data-processing/blob/main/02_global_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Environment setup

In [ ]:
!pip install leafmap geemap==0.16.4 geopandas numpy session_info
import ee, json, geemap, ipyleaflet, os, numpy, backports, session_info
from google.colab import drive
from google.colab import files
!drive.mount('/content/drive')



### Activate Earth Engine

In [ ]:
#!ee.Authenticate() # deprecated in certain Colab environments
!earthengine authenticate
ee.Initialize(project = "murphys-deforisk")

### Jurisidictional boundaries

In [4]:
country = ee.FeatureCollection('FAO/GAUL/2015/level1').filter(
    ee.Filter.equals("ADM0_NAME", "Liberia"))
state = ee.FeatureCollection('FAO/GAUL/2015/level1').filter(
    ee.Filter.equals("ADM0_NAME", "Liberia"))
state_list = country.aggregate_array('ADM1_NAME').distinct().getInfo()
state_list

['Grand Gedeh',
 'River Gee',
 'Gbarpolu',
 'Lofa',
 'Bomi',
 'Bong',
 'Grand Bassa',
 'Grand Cape Mount',
 'Grand Kru',
 'Margibi',
 'Maryland',
 'Montserrado',
 'Nimba',
 'Rivercess',
 'Sinoe']

In [36]:
red = {"color": "red", "width": 2, "lineType": "solid", "fillColor": "00000000"}
white = {"color": "white", "width": 1, "lineType": "solid", "fillColor": "00000000"}
country_label = ee.FeatureCollection([ee.Feature(
    country.geometry().centroid(), {'country_name': country.first().get("ADM0_NAME").getInfo()})])

Map = geemap.Map()
Map.centerObject(country, 6)
Map.add_basemap('Esri.WorldImagery')
Map.addLayer(country.style(**white), {}, "Country")
Map.addLayer(state.style(**white), {}, "States")
Map.add_labels(state,"ADM1_NAME",font_size="7pt",font_color="white",font_family="arial",)
Map.add_labels(country_label, "country_name", font_size="12pt", font_color="white", font_family="arial",)
Map

Map(center=[6.446116651575274, -9.30729708708129], controls=(WidgetControl(options=['position', 'transparent_b…

In [38]:
import pandas as pd

aoi = ee.FeatureCollection(geemap.shp_to_ee('/content/drive/MyDrive/Colab Notebooks/data/AllLandscapes_merge_v02.shp'))
aoi = aoi.filter(ee.Filter.inList('Name', ['Gola Forest National Park', 'Norman', 'Tonglay']))
aoi = aoi.select('Name')

results = []
for feature in aoi.toList(aoi.size()).getInfo():
    name = feature['properties']['Name']
    area_ha = ee.Feature(feature).geometry().area(10).divide(10000).getInfo()
    results.append({'Name': name, 'area_ha': round(area_ha, 4)})  # Round to 4 decimal places

df = pd.DataFrame(results)
total_area = df['area_ha'].sum()
df.loc['Total'] = pd.Series({'Name': 'Total', 'area_ha': total_area})
print(df)

Map = geemap.Map()
Map.centerObject(aoi, 10)  # Center the map on the filtered AOI
Map.add_basemap('Esri.WorldImagery')
Map.addLayer(aoi.style(**white), {}, "AOI")
Map.add_labels(aoi,"Name",font_size="9pt",font_color="red",font_family="arial",font_weight="bold",)
Map


                            Name      area_ha
0      Gola Forest National Park   89411.6539
1                        Tonglay   19676.8689
2                         Norman   12350.6998
Total                      Total  121439.2226


Map(center=[7.544144814296237, -10.758250676400191], controls=(WidgetControl(options=['position', 'transparent…

The Dynamic World dataset provides near real-time (NRT) land cover classifications at a 10-meter resolution. It's important to note that this dataset's availability starts from June 27, 2015. Therefore, we can directly access data for 2019 and potentially 2024 (if data is available up to that year). However, for 2014, we'll need to use the closest available data.



In [40]:
# Load the Dynamic World dataset
dwCol = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")
country_buff = country.geometry().buffer(20000)  # 10000 meters = 10 km

# Filter data for 2019 and 2024
dw2015 = dwCol.filterDate('2015-06-27', '2015-12-31').filterBounds(country_buff)  # Earliest year available
dw2019 = dwCol.filterDate('2019-01-01', '2019-12-31').filterBounds(country_buff)
dw2024 = dwCol.filterDate('2024-01-01', '2024-12-31').filterBounds(country_buff)


# Create composites or mosaics for each year
land_cover_2015 = dw2015.mode().select('label').clip(country_buff)
land_cover_2019 = dw2019.mode().select('label').clip(country_buff)
land_cover_2024 = dw2024.mode().select('label').clip(country_buff)  # Check if data is available

# Define land cover class names (adjust if needed)
land_cover_classes = {
    0: 'Water',
    1: 'Trees',
    2: 'Grass',
    3: 'Flooded Vegetation',
    4: 'Crops',
    5: 'Shrub and Scrub',
    6: 'Built',
    7: 'Bare',
    8: 'Snow and Ice'
}

land_cover_colors = [
    '#419bdf', '#397d49', '#88b053', '#7a87c6',
    '#e49635', '#dfc35a', '#c4281b', '#a59b8f',
    '#b39fe1'
]

vis_params = {
    'min': 0,
    'max': 8,
    'palette': land_cover_colors
}

Map = geemap.Map()
Map.centerObject(country, 6)
Map.add_basemap('Esri.WorldImagery')
Map.addLayer(land_cover_2015, vis_params, 'Land Cover 2015')
Map.addLayer(land_cover_2019, vis_params, 'Land Cover 2019')
Map.addLayer(land_cover_2024, vis_params, 'Land Cover 2024')
legend_dict = {land_cover_classes[i]: land_cover_colors[i] for i in range(len(land_cover_classes))}
Map.add_legend(legend_title="Land Cover Classes", legend_dict=legend_dict)
Map.addLayer(aoi.style(**white), {}, "AOI")
Map

Map(center=[6.446116651575274, -9.30729708708129], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
# Define the years of interest
years = [2014, 2019, 2024]

# Create an empty list to store the results
all_results = []

# Loop through the subzones
for subzone_name in aoi.toList(aoi.size()).map(lambda f: ee.Feature(f).get('Name')).getInfo():
    subzone_aoi = aoi.filter(ee.Filter.eq('Name', subzone_name))

    # Calculate the total area of the subzone
    total_area = subzone_aoi.geometry().area().divide(10000).getInfo()

    # Store subzone results
    subzone_results = {'Land Class': 'Total', 'Total (ha)': total_area}

    # Loop through the years
    for year in years:
        # Get the Dynamic World land cover image for the year
        if year == 2014:
            # Use 2015 as a proxy for 2014
            land_cover_image = dwCol.filterDate('2015-06-27', '2015-12-31').mode().select('label').clip(country)
        else:
            land_cover_image = dwCol.filterDate(f'{year}-01-01', f'{year}-12-31').mode().select('label').clip(country)

        # Calculate area estimates for the current year
        area_estimates = calculate_area_estimates(land_cover_image, subzone_aoi, year)

        # Update subzone results with year-specific estimates
        subzone_results[f'{year} (ha)'] = area_estimates.values()

    all_results.append(subzone_results)

### Export to Drive

Caution, running the following chunk will export 12GB of duplicate rasters to a drive folder.

In [ ]:
from datetime import datetime

# extract pathrow and date from scene ID
def get_pathrow_date(image_collection):
  first_image = image_collection.first()
  scene_id = first_image.get('system:index').getInfo()
  parts = scene_id.split('_')
  pathrow = parts[1]
  date_str = parts[2]
  date_obj = datetime.strptime(date_str, '%Y%m%d')
  date = date_obj.strftime('%Y-%m-%d')
  return pathrow, date

# define export parameters
def define_export_params(image, pathrow, date, band_name):
  description = f'composite_{date}_{band_name if isinstance(band_name, str) else "RGB"}_export'[:100]
  return {
    'image': image.select(band_name),
    'description': description,
    'folder': 'VT0007-deforestation-map',
    'fileNamePrefix': f'LANDSAT_TM-ETM-OLI_{pathrow}_{band_name if isinstance(band_name, str) else "RGB"}_{date}',
    'scale': 30,
    'region': state.geometry(),
    'maxPixels': 1e13,
    'fileFormat': 'GeoTIFF',
    'formatOptions': {'cloudOptimized': True}
  }

# get pathrow and date for each collection
pathrow_2014, date_2014 = get_pathrow_date(collection_2014)
pathrow_2019, date_2019 = get_pathrow_date(collection_2019)
pathrow_2024, date_2024 = get_pathrow_date(collection_2024)

# get band names
bandNames_2014 = composite_2014.bandNames().getInfo()
bandNames_2019 = composite_2019.bandNames().getInfo()
bandNames_2024 = composite_2024.bandNames().getInfo()

# export to cloud bucket
for band_name in bandNames_2014:
    export_params_2014 = define_export_params(composite_2014, pathrow_2014, date_2014, band_name)
    task_2014 = ee.batch.Export.image.toDrive(**export_params_2014)
    task_2014.start()
    print(f"Exporting 2014 image for band {band_name}. Task ID: {task_2014.id}")

for band_name in bandNames_2019:
    export_params_2019 = define_export_params(composite_2019, pathrow_2019, date_2019, band_name)
    task_2019 = ee.batch.Export.image.toDrive(**export_params_2019)
    task_2019.start()
    print(f"Exporting 2019 image for band {band_name}. Task ID: {task_2019.id}")

for band_name in bandNames_2024:
    export_params_2024 = define_export_params(composite_2024, pathrow_2024, date_2024, band_name)
    task_2024 = ee.batch.Export.image.toDrive(**export_params_2024)
    task_2024.start()
    print(f"Exporting 2024 image for band {band_name}. Task ID: {task_2024.id}")

export_params_2014_rgb = define_export_params(composite_2014, pathrow_2014, date_2014, ['RED', 'GREEN', 'BLUE'])
export_params_2014_rgb['image'] = composite_2014.visualize(**rgbVis)
task_2014_rgb = ee.batch.Export.image.toDrive(**export_params_2014_rgb)
task_2014_rgb.start()
print(f"Exporting 2014 RGB image. Task ID: {task_2014_rgb.id}")

export_params_2019_rgb = define_export_params(composite_2019, pathrow_2019, date_2019, ['RED', 'GREEN', 'BLUE'])
export_params_2019_rgb['image'] = composite_2019.visualize(**rgbVis)
task_2019_rgb = ee.batch.Export.image.toDrive(**export_params_2019_rgb)
task_2019_rgb.start()
print(f"Exporting 2019 RGB image. Task ID: {task_2019_rgb.id}")

export_params_2024_rgb['image'] = composite_2024.visualize(**rgbVis)
task_2024_rgb = ee.batch.Export.image.toDrive(**export_params_2024_rgb)
task_2024_rgb.start()
print(f"Exporting 2024 RGB image. Task ID: {task_2024_rgb.id}")

Exporting 2014 RGB image. Task ID: IIMNP6IZ4ALUYMXEILXGCRND


In [ ]:
session_info.show()